In [2]:
# Ex. 1 ----------------------------------------------------------------------------------------------------------------------
import pandas as pd
from sklearn.naive_bayes import BernoulliNB
import numpy as np

# Create the training set
features = ['study', 'free', 'money']
target = 'is_spam'
messages = pd.DataFrame(
[(1, 0, 0, 0),
(0, 0, 1, 0),
(1, 0, 0, 0),
(1, 1, 0, 0)] +
[(0, 1, 0, 1)] * 4 +
[(0, 1, 1, 1)] * 4,
columns=features+[target])

# Create the prediction set
X = messages[features]
y = messages[target]
cl = BernoulliNB(alpha=1e-10).fit(X, y)

print(cl.class_log_prior_)
print(cl.feature_log_prob_)

[-1.09861229 -0.40546511]
[[-2.87682072e-01 -1.38629436e+00 -1.38629436e+00]
 [-2.51052925e+01 -1.24997790e-11 -6.93147181e-01]]


In [3]:
# a)
def class_log_prior_calculator(messages):
    y = messages[target]
    prob_0 = 0
    prob_1 = 0
    for element in y:
        if element == 0:
            prob_0 += 1 
        elif element == 1:
            prob_1 += 1
    prob_0 = prob_0 / len(y)
    prob_1 = prob_1 / len(y)
    print(f"class_log_prior with no smoothing: {np.log(prob_0)}, {np.log(prob_1)}")

class_log_prior_calculator(messages)

class_log_prior with no smoothing: -1.0986122886681098, -0.40546510810816444


In [4]:
# b)
import numpy as np
import pandas as pd

def feature_log_prob_calculator(messages):
    features = messages.columns[:-1]  # Assuming the target variable is the last column
    target_column = messages.columns[-1]

    study_spam = 0
    study_not_spam = 0
    free_spam = 0
    free_not_spam = 0
    money_spam = 0
    money_not_spam = 0
    y_is_0 = 0
    y_is_1 = 0

    for i in range(len(messages)):
        if messages[target_column][i] == 0:
            y_is_0 += 1
            if messages[features[0]][i] == 1:
                study_spam += 1
            if messages[features[1]][i] == 1:
                free_spam += 1
            if messages[features[2]][i] == 1:
                money_spam += 1
        elif messages[target_column][i] == 1:
            y_is_1 += 1
            if messages[features[0]][i] == 1:
                study_not_spam += 1
            if messages[features[1]][i] == 1:
                free_not_spam += 1
            if messages[features[2]][i] == 1:
                money_not_spam += 1

    print(f"feature_log_prob with no smoothing: {np.log(study_spam/y_is_0)}, {np.log(free_spam/y_is_0)}, {np.log(money_spam/y_is_0)}, {np.log(study_not_spam/y_is_1)}, {np.log(free_not_spam/y_is_1)}, {np.log(money_not_spam/y_is_1)} ")

feature_log_prob_calculator(messages)


feature_log_prob with no smoothing: -0.2876820724517809, -1.3862943611198906, -1.3862943611198906, -inf, 0.0, -0.6931471805599453 


C:\Users\Maria\AppData\Local\Temp\ipykernel_17220\2666171953.py:36: RuntimeWarning: divide by zero encountered in log
  print(f"feature_log_prob with no smoothing: {np.log(study_spam/y_is_0)}, {np.log(free_spam/y_is_0)}, {np.log(money_spam/y_is_0)}, {np.log(study_not_spam/y_is_1)}, {np.log(free_not_spam/y_is_1)}, {np.log(money_not_spam/y_is_1)} ")


In [24]:
# Ex. 3 ----------------------------------------------------------------------------------------------------------------------
# a)
import pandas as pd
from tools.pd_helpers import apply_counts

d = pd.DataFrame({'X1': [0, 0, 1, 1, 0, 0, 1, 1],
                  'X2': [0, 0, 0, 0, 1, 1, 1, 1],
                  'C' : [2, 18, 4, 1, 4, 1, 2, 18],
                  'Y' : [0, 1, 0, 1, 0, 1, 0, 1]})
d=apply_counts(d, 'C')

In [31]:
class BernoulliJB:
    def fitJB(self, X, y):
        self.classes, y_counts = np.unique(y, return_counts=True)
        self.class_probs = y_counts / len(y)
        
        self.feature_probs = {}
        for c in self.classes:
            class_data = X[y == c]
            feature_probs_c = class_data.sum(axis=0) / len(class_data)
            self.feature_probs[c] = feature_probs_c

    def predict_proba(self, X):
        probabilities = []
        for c in self.classes:
            probs_c = self.class_probs[c] * np.prod(X * self.feature_probs[c] + (1 - X) * (1 - self.feature_probs[c]), axis=1)
            probabilities.append(probs_c)
        
        probabilities = np.array(probabilities).T
        probabilities /= probabilities.sum(axis=1, keepdims=True)
        
        return probabilities

In [33]:
# b)
print("Joint Bayes estimates 2^(n+1) - 1 probabilities, where n is the number of input attributes.")

Joint Bayes estimates 2^(n+1) - 1 probabilities, where n is the number of input attributes.


In [32]:
# c)
jb_model = BernoulliJB()

X_train = d[['X1', 'X2']].values
y_train = d['Y'].values

jb_model.fitJB(X_train, y_train)
X_instance = np.array([[0, 0]])
proba_instance = jb_model.predict_proba(X_instance)
print(proba_instance)

[[0.24 0.76]]


In [22]:
# d)
nb_model = BernoulliNB(alpha=0) 
nb_model.fit(X_train, y_train)
X_instance = np.array([[0, 0]])
proba_instance_nb = nb_model.predict_proba(X_instance)

print(proba_instance_nb)

[[0.24 0.76]]


c:\users\maria\appdata\local\programs\python\python39\lib\site-packages\sklearn\naive_bayes.py:624: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
c:\users\maria\appdata\local\programs\python\python39\lib\site-packages\sklearn\naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
